# Day 24

https://adventofcode.com/2021/day/24

In [163]:
def readInput24(filename):
    with open(filename) as f:
        return [ l.split(" ") for l in f.read().strip('\n').split('\n') ]

In [164]:
class ALU:
    def __init__(self):
        self.v = {}
        self.v['w'] = 0
        self.v['x'] = 0 
        self.v['y'] = 0
        self.v['z'] = 0
        
    def loadInputs(self,inputs):
        self.inputs = inputs
        self.index = 0
      
    def loadProg(self,prog):
        self.prog = prog
    
    def inp(self,a):
        self.v[a] = self.inputs[self.index]
        self.index += 1
    
    def add(self,a,b):
        try:
            self.v[a] += int(b)
        except:
            self.v[a] += self.v[b]
    
    def mul(self,a,b):
        try:
            self.v[a] *= int(b)
        except:
            self.v[a] *= self.v[b]
            
    def div(self,a,b):
        try:
            self.v[a] //= int(b)
        except:
            self.v[a] //= self.v[b]
            
    def mod(self,a,b):
        try:
            self.v[a] %= int(b)
        except:
            self.v[a] %= self.v[b]   
            
    def eql(self,a,b):
        try:
            self.v[a] = int( self.v[a]==int(b) )
        except:
            self.v[a] = int( self.v[a]==self.v[b] )
    
    operation = {
        'inp': inp,
        'add': add,
        'mul': mul,
        'div': div,
        'mod': mod,
        'eql': eql,
    }
    
    def run(self,j=0,verbose=False):
        while True:
            p = self.prog[j]
            if verbose:
                print(p)
            if p[0]=='inp':
                 ALU.operation[p[0]](self,p[1])
            else:
                ALU.operation[p[0]](self,p[1],p[2])
            j += 1
            if j>=len(self.prog):
                break

In [165]:
prog = readInput24("data/day24test1.txt")

alu = ALU()
alu.loadProg(prog)
alu.loadInputs([1])
alu.run()
print(alu.v['x'])

-1


In [166]:
prog = readInput24("data/day24test2.txt")

alu = ALU()
alu.loadProg(prog)
alu.loadInputs([1,2])
alu.run()
print(alu.v['z'])

alu.loadInputs([1,3])
alu.run()
print(alu.v['z'])

0
1


In [167]:
prog = readInput24("data/input24.txt")

serial = "13579246899999"

inputs = [ int(c) for c in list(serial) ] 

alu = ALU()
alu.loadProg(prog)
alu.loadInputs(inputs)
alu.run()
print(alu.v['z'])

5427581560


### Implementing ALU as sequence of sub-programs

Each subprogram runs on a digit starting from a given state left from previous part of program.

In [6]:
prog = readInput24("data/input24.txt")

# separate program sections acting on different digits
progDigit = []
l = 18
for d in range(14):
    progDigit.append(prog[d*l:(d+1)*l])

In [100]:
alu = ALU()

def Z(d=0,i=9,v=(0,0,0,0)):
    # load previous status on registers
    alu.v['w'] = v[0]
    alu.v['x'] = v[1]
    alu.v['y'] = v[2]
    alu.v['z'] = v[3]
    # load subprogram corresponding to current digit
    alu.loadProg(progDigit[d])
    # load input digit value
    alu.loadInputs([i])
    alu.run()
    return (alu.v['w'],alu.v['x'],alu.v['y'],alu.v['z'])

def MONADcache(digits):
    v = (0,0,0,0)
    for i in range(14):
        v = Z(i,digits[i],v)
    return v[3]

def MONAD(digits):
    alu = ALU()
    alu.loadProg(prog)
    alu.loadInputs(digits)
    alu.run()
    return alu.v['z']

digits = [9,9,9,9,1,9,9,9,9,9,7,9,9,9]

MONAD(inputs), MONADcache(inputs)

(5427581560, 5427581560)

### Reverse-engineering instruction blocks

Each instruction block uses the same set of operations with the current digit and the `z` register value as input (starting from 0), and different "constants" used by the operations:

In [168]:
progDigit[3]

[['inp', 'w'],
 ['mul', 'x', '0'],
 ['add', 'x', 'z'],
 ['mod', 'x', '26'],
 ['div', 'z', '26'],
 ['add', 'x', '-5'],
 ['eql', 'x', 'w'],
 ['eql', 'x', '0'],
 ['mul', 'y', '0'],
 ['add', 'y', '25'],
 ['mul', 'y', 'x'],
 ['add', 'y', '1'],
 ['mul', 'z', 'y'],
 ['mul', 'y', '0'],
 ['add', 'y', 'w'],
 ['add', 'y', '12'],
 ['mul', 'y', 'x'],
 ['add', 'z', 'y']]

In [8]:
a = [ int(progDigit[i][4][2])  for i in range(14) ]
b = [ int(progDigit[i][5][2])  for i in range(14) ]
c = [ int(progDigit[i][15][2]) for i in range(14) ]

a, b, c

([1, 1, 1, 26, 26, 1, 1, 26, 1, 1, 26, 26, 26, 26],
 [11, 12, 13, -5, -3, 14, 15, -16, 14, 15, -7, -11, -6, -11],
 [16, 11, 12, 12, 12, 2, 11, 4, 12, 9, 10, 11, 6, 15])

In [56]:
def progBlock_(z, w, a, b, c):
    # a = 1 or 26
    # b = constant added to x register
    # c = constant added to y register
    znext = z//a
    if w==(z%26+b): # x=0
        return znext
    else: # x=1
        return 26*znext + w+c

def progBlock(z, w, i):
    # a = 1 or 26
    # b = constant added to x register
    # c = constant added to y register
    znext = z//a[i]
    if w==(z%26+b[i]): # x=0
        return znext
    else: # x=1
        return 26*znext+w+c[i]

def MONADblocks(digits):
    z = 0
    for i in range(14):
        z = progBlock(z,digits[i],i)
    return z

MONAD(inputs), MONADblocks(inputs)

(5427581560, 5427581560)

### Tracking intermediate state of `z` register to reduce search space

Given the repetitive form of the various algorythm blocks, I suspect the intermediate states of the register `z`  will repeat themselves, so I probably don't need to compute $9^{14}$ combinations to brute force the solution, but a (hopefully very much) reduced set (it works!!).

Saving list of `w,z` values prodicing a given new state of the `z` register to allow a "back-navigation" to build the solution from last digit.

In [139]:
from collections import defaultdict
zMap = []
zMap.append(defaultdict(list))
zMap[0][0] = []

for i in range(14):
    print("Digit {:2d} ... ".format(i),end="")
    zMapNext = defaultdict(list)
    for z in zMap[i].keys():
        for w in range(1,10):
            zNext = progBlock(z,w,i)
            zMapNext[zNext].append((w,z)) # saving w,z to allow back-navigation to "build" the solution from last digit
    print("unique z states: {} ".format(len(zMapNext.keys())))
    zMap.append(zMapNext)

Digit  0 ... unique z states: 9 
Digit  1 ... unique z states: 81 
Digit  2 ... unique z states: 729 
Digit  3 ... unique z states: 810 
Digit  4 ... unique z states: 819 
Digit  5 ... unique z states: 7371 
Digit  6 ... unique z states: 66339 
Digit  7 ... unique z states: 73710 
Digit  8 ... unique z states: 663390 
Digit  9 ... unique z states: 5970510 
Digit 10 ... unique z states: 6582303 
Digit 11 ... unique z states: 6700239 
Digit 12 ... unique z states: 7035786 
Digit 13 ... unique z states: 6862772 


In [159]:
digits = []
ztarget = 0
for i in range(14,0,-1):
    dmax,zmax = max(zMap[i][ztarget])
    digits.append(dmax)
    print("Digit {:2d} = {} -> previous z value = {}".format(i,dmax,zmax))
    ztarget = zmax
digits = digits[::-1]

print("\nPart 1:","".join([str(d) for d in digits]))

Digit 14 = 9 -> previous z value = 20
Digit 13 = 5 -> previous z value = 531
Digit 12 = 9 -> previous z value = 13826
Digit 11 = 9 -> previous z value = 359492
Digit 10 = 7 -> previous z value = 13826
Digit  9 = 8 -> previous z value = 531
Digit  8 = 4 -> previous z value = 13826
Digit  7 = 9 -> previous z value = 531
Digit  6 = 9 -> previous z value = 20
Digit  5 = 9 -> previous z value = 532
Digit  4 = 9 -> previous z value = 13846
Digit  3 = 2 -> previous z value = 532
Digit  2 = 1 -> previous z value = 20
Digit  1 = 4 -> previous z value = 0

Part 1: 41299994879959


In [161]:
digits = []
ztarget = 0
for i in range(14,0,-1):
    dmin,zmin = min(zMap[i][ztarget])
    digits.append(dmin)
    print("Digit {:2d} = {} -> previous z value = {}".format(i,dmin,zmin))
    ztarget = zmin
digits = digits[::-1]

print("\nPart 2:","".join([str(d) for d in digits]))

Digit 14 = 6 -> previous z value = 17
Digit 13 = 1 -> previous z value = 449
Digit 12 = 2 -> previous z value = 11687
Digit 11 = 3 -> previous z value = 303872
Digit 10 = 1 -> previous z value = 11687
Digit  9 = 1 -> previous z value = 449
Digit  8 = 1 -> previous z value = 11691
Digit  7 = 6 -> previous z value = 449
Digit  6 = 5 -> previous z value = 17
Digit  5 = 9 -> previous z value = 454
Digit  4 = 8 -> previous z value = 11817
Digit  3 = 1 -> previous z value = 454
Digit  2 = 1 -> previous z value = 17
Digit  1 = 1 -> previous z value = 0

Part 2: 11189561113216
